In [ ]:
!mkdir /root/.kaggle/
!pip install pandas
import os
import json

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns

from sklearn.model_selection import train_test_split
from plotly.subplots import make_subplots
from scipy.stats import pearsonr
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import accuracy_score, f1_score

from sklearn import metrics
 
%matplotlib inline
 
 
# Installing the Kaggle package
!pip install kaggle 
 
#Important Note: complete this with your own key - after running this for the first time remmember to **remove** your API_KEY
api_token = {"username":"roypaz","key":"fe3bd5d09b6406f78a557680670c662b"}
 
# creating kaggle.json file with the personal API-Key details 
# You can also put this file on your Google Drive
with open('/root/.kaggle/kaggle.json', 'w') as file:
  json.dump(api_token, file)
!chmod 600 /root/.kaggle/kaggle.json

mkdir: cannot create directory ‘/root/.kaggle/’: File exists


In [ ]:
!kaggle datasets download -d shivam2503/diamonds -p ./datasets/diamonds/
!unzip ./datasets/diamonds/*.zip  -d ./datasets/diamonds/

diamonds_df = pd.read_csv('/content/datasets/diamonds/diamonds.csv')

diamonds.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  ./datasets/diamonds/diamonds.zip
replace ./datasets/diamonds/diamonds.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


## 1

In [ ]:
train_df, test_df = train_test_split(diamonds_df, test_size=0.2)
knn_classifier1 = KNeighborsClassifier(n_neighbors=1)
featues_list = ['carat', 'depth', 'table', 'x', 'y', 'price']
knn_classifier1.fit(train_df[featues_list], train_df["clarity"].values)
actual_clarity_list = test_df["clarity"].values
predict_clarity_list1 = knn_classifier1.predict(test_df[featues_list])
predict_clarity_list1

array(['SI1', 'SI1', 'VS1', ..., 'VVS1', 'SI1', 'VVS1'], dtype=object)

In [ ]:
f1score1 = metrics.f1_score(actual_clarity_list, predict_clarity_list1, average='weighted') #check micro
accuracy1 = metrics.accuracy_score(actual_clarity_list, predict_clarity_list1, normalize=True)
print(f'f1 score: {f1score1}\naccuracy score: {accuracy1}')

f1 score: 0.354717711886582
accuracy score: 0.35511679644048944


## 2

In [ ]:
xpar = ['carat', 'table', 'price', 'depth', 'y', 'x']
train, test = train_test_split(diamonds_df, test_size = 0.2)
def knn_class(k=1):
  knn = KNeighborsClassifier(n_neighbors=k)
  knn.fit(train[xpar], train['clarity'].values)
  real_val = test.clarity.values
  pridicted = knn.predict(test[xpar])
  cm = metrics.confusion_matrix(real_val, pridicted)
  return cm, real_val, pridicted

cm, real_val, pridicted = knn_class(1)

In [ ]:
x = [i for i in range(1,50,2)]
df = pd.DataFrame()
df['k'] = x
df['accuracy'] = df['k'].apply(lambda n: accuracy_score(*knn_class(k=n)[1:]))
df['f1'] = df['k'].apply(lambda n: f1_score(*knn_class(k=n)[1:], average = 'macro'))

df.set_index("k", inplace=False)

subfig = make_subplots(specs=[[{"secondary_y": True}]])

# create two independent figures with px.line each containing data from multiple columns
fig = px.line(df, x=df.k, y=df['accuracy'], render_mode="svg")
fig2 = px.line(df, x=df.k, y=df['f1'], render_mode="svg")

fig.update_traces(mode='markers+lines')
fig2.update_traces(yaxis="y2", mode='markers+lines')

subfig.add_traces(fig.data + fig2.data)
subfig.layout.xaxis.title="<b>K"
subfig.layout.yaxis.title="<b>Accuracy"
subfig.layout.yaxis2.title="<b>f1_score"
subfig.for_each_trace(lambda t: t.update(line=dict(color=t.marker.color)))

subfig.show()

## 3

In [ ]:
from sklearn import preprocessing
label_colums = ['color','clarity']
for col in label_colums:
  le = preprocessing.LabelEncoder()
  diamonds_df[f'numeric_{col}']= le.fit_transform(diamonds_df[col])

In [ ]:
from sklearn import tree


X_train, X_test, y_train, y_test = train_test_split(diamonds_df, diamonds_df['cut'].values, test_size = 0.2)

xpar = ['carat', 'table', 'price', 'depth', 'y', 'x']
best = 0
bst_k = 0
for k in [1, 3, 5]:
  knn = KNeighborsClassifier(n_neighbors=k)
  knn.fit(X_train[xpar], y_train)
  real_val = y_test
  pridicted = knn.predict(X_test[xpar])
  res = accuracy_score(real_val, pridicted)
  if res > best:
    best = res
    bst_k = k


tree_clf = tree.DecisionTreeClassifier()
tree_clf = tree_clf.fit(X_train[xpar], y_train)
pridicted = tree_clf.predict(X_test[xpar])
tree_res = accuracy_score(y_test, pridicted)

print(f"the best result was {tree_res}, with decision tree model" if tree_res > best else f"the best result was {best}, with {bst_k}-nn model")

the best result was 0.7157953281423804, with decision tree model


## 4

In [ ]:
from sklearn.preprocessing import LabelEncoder

new_col = ['numeric_color', 'numeric_clarity']
xpar = ['carat', 'table', 'price', 'depth', 'y', 'x'] + new_col
best2 = 0
bst_k2 = 0
for k in [1, 3, 5]:
  knn = KNeighborsClassifier(n_neighbors=k)
  knn.fit(X_train[xpar], y_train)
  real_val = y_test
  pridicted = knn.predict(X_test[xpar])
  res = accuracy_score(real_val, pridicted)
  print(k, res)
  if res > best2:
    best2 = res
    bst_k2 = k


tree_clf = tree.DecisionTreeClassifier()
tree_clf = tree_clf.fit(X_train[xpar], y_train)
pridicted = tree_clf.predict(X_test[xpar])
tree_res2 = accuracy_score(y_test, pridicted)
print('tree', tree_res2)

print(f" KNN accuracy increased with k = {bst_k2}" if best2 > best else " KNN accuracy was better before..","\n",
      f"Decision Tree has now better accuracy" if tree_res2 > tree_res else "Decision Tree accuracy was better before..." )

1 0.5324434556915091
3 0.5265109380793475
5 0.5324434556915091
tree 0.7068038561364479
 KNN accuracy was better before.. 
 Decision Tree accuracy was better before...


## 5

In [ ]:
precentage = [0.05,0.1,0.2,0.5,0.7,0.8,0.9]
predict_list4 = []
accuracy_cut = []
train_df, test_df = train_test_split(diamonds_df, test_size= 0.2)
for p in precentage:
  # temp_df = diamonds_df.iloc[:int(len(diamonds_df)*p)]
  train_precent = train_df.iloc[:int(len(train_df)*p)]
  actual_cut_list = test_df["cut"].values
  knn_classifier = KNeighborsClassifier(n_neighbors=3) #  k = 3
  knn_classifier.fit(train_precent[featues_list], train_precent["cut"].values)
  predict_list4.append(knn_classifier.predict(test_df[featues_list]))
for i,j in zip(predict_list4,precentage):
  accuracy_cut_temp = metrics.accuracy_score(actual_cut_list, i, normalize=True)
  accuracy_cut.append(accuracy_cut_temp)

In [ ]:
df2 = pd.DataFrame()
df2['percentage'] = precentage
df2['accuracy'] = accuracy_cut
df2.set_index("accuracy", inplace=False)
fig2 = px.line(df2, x=df2['percentage'], y=df2['accuracy'], render_mode="svg")
fig2.update_xaxes(title_text="<b>trainset precentage")
fig2.update_yaxes(title_text="<b>accuracy")
fig2.update_traces(mode='markers+lines')
fig2.show()